In [31]:
import pandas as pd
import numpy as np

The Simple Recommender

In [32]:
df = pd.read_csv('\computer vision\datasets\movies_metadata.csv.xls')

C:\Users\mchan\AppData\Local\Temp\ipykernel_1840\3467934803.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('\computer vision\datasets\movies_metadata.csv.xls')


In [33]:
df

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


In [34]:
df.shape

(45466, 24)

In [35]:
df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

we want to consider
   -choosing a metric
   - decide on the prerequisites for the movie to be the featured on the chart
   - calculate the sore for every movie that satisfies the conditions
-  - output the list of the movies in decreasing order of their scores 

The Metric

- the metric is the numeric quantity on which we rank the movies

rating : lets say a movie with 8/10 with viewers 10000 and a movie with 9.5/10 with viewers 100 cannot be taken into considerations just by rating

we need a metric which can, to an extend take into account both the movie rating and the no of votes(popularity)

-this would give a greater preference to a blockbuster movie rated 8 by 10000 user over an art movie rated 9 by 100 users.

-movie with few ratings are not much reliable
eg - A movie rated 10/10 by 5 users does not necessarily means that it is agood movie.

-here we are using imdb's weighted rating formula

THE PREREQUISITES

-There is no fixed value for m(it is an arbitrary value)
-For our purpose we are using 80th precentile movie as our value for m.
-For a movie to be considered in the rankings it must have garnered more votes than atleast 80% of the movies present in our dataset

In [36]:
ta_movie = df[df['original_language'] == 'ta']

In [37]:
df[df['original_language'] == 'ta']

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
13867,False,NaN,500000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,69426,tt0242572,ta,Kandukondain Kandukondain,Two young women with differing views on love f...,...,2000-05-05,500000.0,157.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Kandukondain Kandukondain,False,6.4,7.0
15232,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10769, 'n...",NaN,40998,tt0312859,ta,கன்னத்தில் முத்தமிட்டாள்,A little girl is told by her parents that she ...,...,2002-02-14,0.0,123.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Kannathil Muthamittal,False,7.4,8.0
17878,False,NaN,26400000,"[{'id': 28, 'name': 'Action'}, {'id': 35, 'nam...",NaN,148284,tt1305797,ta,எந்திரன்,"Dr. Vasi invents a super-powered robot, Chitti...",...,2010-10-01,62000000.0,155.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Enthiran,False,6.5,46.0
18983,False,NaN,0,"[{'id': 80, 'name': 'Crime'}, {'id': 53, 'name...",http://www.unnaipoloruvan.com,25121,tt1417299,ta,உன்னைப்போல் ஒருவன்,A common man tries to fight terrorism by makin...,...,2009-09-18,0.0,120.0,"[{'iso_639_1': 'ta', 'name': 'தமிழ்'}]",Released,Someone Like You,Unnaipol Oruvan,False,6.9,10.0
20412,False,NaN,7500000,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,147815,tt2344672,ta,கடல்,A wrongdoer is caught red-handed and he is not...,...,2013-02-01,0.0,164.0,"[{'iso_639_1': 'ta', 'name': 'தமிழ்'}, {'iso_6...",Released,NaN,Kadal,False,5.9,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44917,False,NaN,0,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",NaN,449742,tt6712026,ta,கவண்,"A budding journalist passionate about ethical,...",...,2017-03-31,0.0,160.0,"[{'iso_639_1': 'ta', 'name': 'தமிழ்'}]",Released,NaN,Kavan,False,7.3,7.0
45247,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,66247,tt0140399,ta,Muthu,Muthu starring by rajinikanth and meena. A lav...,...,1995-05-05,0.0,180.0,"[{'iso_639_1': 'ml', 'name': ''}, {'iso_639_1'...",Released,NaN,Muthu,False,6.3,10.0
45248,False,NaN,0,"[{'id': 28, 'name': 'Action'}]",NaN,49074,tt0139876,ta,Baasha,Auto rickshaw driver Manickam avoids violence ...,...,1995-01-15,15000000.0,145.0,"[{'iso_639_1': 'ta', 'name': 'தமிழ்'}]",Released,NaN,Baasha,False,7.8,14.0
45249,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10769, 'n...",NaN,47940,tt0213969,ta,படையப்பா,Padayappa is a 1999 Tamil film written and dir...,...,1999-04-01,0.0,172.0,"[{'iso_639_1': 'ta', 'name': 'தமிழ்'}]",Released,NaN,Padayappa,False,7.3,13.0


In [38]:
#calculate the no of votes garnered by the 80th percentile movie
m = ta_movie['vote_count'].quantile()
m

10.0

-Consider movie only more than 45 minutes and less than 300 minutes

In [39]:
#movies >45 and <300
q_movies = ta_movie[(ta_movie['runtime'] >= 45) & (ta_movie['runtime'] < 300)]
q_movies = q_movies[q_movies['vote_count'] >= m]

In [40]:
ta_movie.shape

(78, 24)

In [41]:
q_movies.shape

(40, 24)

CACULATE THE SCORE

In [42]:
c = ta_movie['vote_average'].mean()

In [43]:
c

6.6923076923076925

In [44]:
#function to calculate the imdb weighted rating for each movie
def weighted_rating(x,m=m,c=c):
    v = x['vote_count']
    R = x['vote_average']
    #compute the weighted rating
    return (v/(v+m)*R) + (m/(m+v) * c)

In [45]:
q_movies['score'] = q_movies.apply(weighted_rating,axis=1)

In [46]:
q_movies

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,score
17878,False,NaN,26400000,"[{'id': 28, 'name': 'Action'}, {'id': 35, 'nam...",NaN,148284,tt1305797,ta,எந்திரன்,"Dr. Vasi invents a super-powered robot, Chitti...",...,62000000.0,155.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Enthiran,False,6.5,46.0,6.534341
18983,False,NaN,0,"[{'id': 80, 'name': 'Crime'}, {'id': 53, 'name...",http://www.unnaipoloruvan.com,25121,tt1417299,ta,உன்னைப்போல் ஒருவன்,A common man tries to fight terrorism by makin...,...,0.0,120.0,"[{'iso_639_1': 'ta', 'name': 'தமிழ்'}]",Released,Someone Like You,Unnaipol Oruvan,False,6.9,10.0,6.796154
20695,False,NaN,3900000,"[{'id': 18, 'name': 'Drama'}, {'id': 10769, 'n...",NaN,60579,tt0222012,ta,Hey Ram,Saketh Ram's wife is raped and killed during d...,...,4900000.0,199.0,"[{'iso_639_1': 'hi', 'name': 'हिन्दी'}, {'iso_...",Released,An Experiment With Truth.,Hey Ram,False,6.7,10.0,6.696154
30522,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,264038,tt3973410,ta,Kaaka Muttai,Two slum kids yearn to taste a pizza after bei...,...,0.0,109.0,"[{'iso_639_1': 'ta', 'name': 'தமிழ்'}]",Released,NaN,Kaaka Muttai,False,8.2,12.0,7.514685
30565,False,NaN,240000,"[{'id': 53, 'name': 'Thriller'}, {'id': 35, 'n...",NaN,139357,tt2585562,ta,Pizza,The story is about how around a pizza delivery...,...,2800000.0,127.0,"[{'iso_639_1': 'ta', 'name': 'தமிழ்'}, {'iso_6...",Released,NaN,Pizza,False,5.5,11.0,6.067766
30643,False,NaN,17000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,117319,tt2199711,ta,விஸ்வரூபம்,"Vishwanathan, a Kathak dance teacher in New Yo...",...,51000000.0,147.0,"[{'iso_639_1': 'hi', 'name': 'हिन्दी'}, {'iso_...",Released,NaN,Vishwaroopam,False,5.5,25.0,5.840659
31790,False,NaN,310000,"[{'id': 35, 'name': 'Comedy'}, {'id': 53, 'nam...",NaN,189227,tt2877108,ta,Soodhu Kavvum,Dass specialises in small time kidnapping and ...,...,12000000.0,138.0,"[{'iso_639_1': 'ta', 'name': 'தமிழ்'}]",Released,Evil Engulfs,Soodhu Kavvum,False,7.0,11.0,6.853480
31800,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 53, 'nam...",NaN,329134,tt4328798,ta,மாஸ் என்கிற மாசிலாமணி,A conman gains the power to see and communicat...,...,0.0,151.0,"[{'iso_639_1': 'ta', 'name': 'தமிழ்'}]",Released,NaN,Massu Engira Maasilamani,False,4.9,12.0,5.714685
31801,False,NaN,0,"[{'id': 53, 'name': 'Thriller'}, {'id': 28, 'n...",NaN,55951,tt1483820,ta,Aadhavan,Aadhavan (Tamil: ஆதவன்) is a 2009 Tamil action...,...,0.0,135.0,"[{'iso_639_1': 'ta', 'name': 'தமிழ்'}]",Released,ஆதவன்,Aadhavan,False,4.4,10.0,5.546154
32707,False,NaN,0,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",NaN,330421,tt4429128,ta,Papanasam,A cinephile tries to protect his family by hid...,...,14000000.0,181.0,"[{'iso_639_1': 'ta', 'name': 'தமிழ்'}]",Released,NaN,Papanasam,False,7.4,13.0,7.092308


SORTING AND OUTPUT

In [47]:
q_movies.sort_values(by='score',ascending=False,inplace = True)

In [48]:
q_movies[['title','vote_count','vote_average','score','runtime']]

,title,vote_count,vote_average,score,runtime
30522,Kaaka Muttai,12.0,8.2,7.514685,109.0
38257,Thani Oruvan,21.0,7.8,7.442680,159.0
37509,Vinnaithaandi Varuvaayaa,19.0,7.7,7.352520,157.0
45248,Baasha,14.0,7.8,7.338462,145.0
43095,Nayakan,15.0,7.5,7.176923,145.0
33173,Vaaranam Aayiram,13.0,7.5,7.148829,169.0
32707,Papanasam,13.0,7.4,7.092308,181.0
45249,Padayappa,13.0,7.3,7.035786,172.0
34537,Jigarthanda,11.0,7.3,7.010623,170.0
43199,24,22.0,7.1,6.972596,164.0


Tamil top movie recommendation